# TP MODELISATION DU CHURN

### Rechargement des données préparées

In [2]:
import os
repertoire = "C:\\Users\\dgr\\OneDrive - Publicis Groupe\\Documents\\Formations\\SCU Python\\NEW\\data"
os.chdir(repertoire)
import pickle
file=open("churn_prepared.pydata","rb")
Xy=pickle.load(file)
file.close()

### Charger les librairies

In [4]:
import sys
sys.path.append("C:\\Users\\dgr\\OneDrive - Publicis Groupe\\Documents\\Formations\\SCU Python\\NEW\\modules")
sys.path

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from fonctions_metrics import lift
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,roc_auc_score,roc_curve,auc

import matplotlib.pyplot as plt

### PARAMETRER LE MODELE REG PENALISEE
Choisir ridge (L2) ou lasso (L1) ou elasticnet, avec la classe LogisticRegression
Définir une grille d'hyper param pour C
et utliser GridSearchCV pour la balayer

In [ ]:
## Regression logistique avec pénalité lasso et grid search
#### on cherche par CV le meilleur C (1/alpha) le coef de regularisation

In [ ]:
param = [{"C":(....)}]
modeleLassoCV = GridSearchCV (....,n_jobs=-1,scoring='roc_auc')

modeleLassoCV = modeleLassoCV.fit(X_train,y_train)

modeleLassoCV.best_params_
#{'C': 0.1}

In [ ]:
# design du modele et app
model_LogitL1 =LogisticRegression( ... ).fit(X_train,y_train)

In [ ]:
# récupérer les coef
coef=list(model_LogitL1.coef_[0]) # c un array2d, 
len(coef) # il y a 20 

In [ ]:
# combien d'élemnts non nuls ?
feature_0= list(map(lambda x: x==0.0, coef)) #print(feature_0)
feature_0.count(True) # 6
del feature_0

# PLus on baisse C, plus de colonnes s'annulent (0)
# print(pd.DataFrame({'Coefficients': list(reg.coef_)}, list(X.columns.values)))

In [ ]:
# Prediction des probabilités de 1 avec    model_LogitL1.predict_proba(X)
probas_test = ...
probas_train = ...

In [ ]:
# score: accuracy taux de bien classé global au cutoff de 0.5
model_LogitL1.score(X_test, y_test)
model_LogitL1.score(X_train,y_train)

In [ ]:
#AUC
roc_auc_score(y_train,probas_train)
roc_auc_score(y_test,probas_test)

In [ ]:
from fonctions_metrics import lift
#compute lift at 10%#
lift(probas_train,X_train,y_train)
lift(probas_test,X_test,y_test)

In [ ]:
#compute lift at 5%
lift(probas_train,X_train,y_train,p=5)
lift(probas_test,X_test,y_test,p=5)

In [ ]:
# ENR LES RESULTATS DANS UN DICTIONNAIRE POUR POUVOIR COMPARER
model='Score Lasso'
# métriques (liste de dictionnaires)
metriques = [{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),2),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}]


In [ ]:
# mat de confusion à un cutoff donné
pred = (probas_test > 0.14).astype(int)
print(confusion_matrix(y_test, pred))
# parmi les vrais 1 82% sont bien prédits
print(pd.crosstab(y_test,pred).apply(lambda r: r/r.sum(), axis=1))
# parmi les predits 1 34% sont des vrais positifs
print(pd.crosstab(y_test,pred).apply(lambda r: r/r.sum(), axis=0))

target_names = ['Fidèles','Churners']

# métriques au cutoff donné
print(classification_report(y_test,pred, target_names=target_names))
# métriques au cutoff par défaut
print(classification_report(y_test,model_LogitL1.predict(X_test), target_names=target_names))



### PARAMETRER LE MODELE REG PENALISEE ELASTICNET

In [ ]:
#### Régression logistique elastic net : SGDClassifier : loss = log --> logistique
#### penalty = "elasticnet"
#### alpha entre 1 et 1000 , et l1_ratio entre 0 et 1
from sklearn.linear_model import SGDClassifier
#Defaults to ‘hinge’, which gives a linear SVM. The ‘log’ loss gives logistic regression, a probabilistic classifier. 
#l1_ratio : float
#The Elastic Net mixing parameter, with 0 <= l1_ratio <= 1. l1_ratio=0 corresponds to L2 penalty, l1_ratio=1 to L1. Defaults to 0.15

param = [{"alpha":(0.001,0.01,0.025,0.05,0.1,0.2)} ]
sgdElasticAUCnow = GridSearchCV (SGDClassifier(loss="log", penalty="elasticnet",l1_ratio=0.25,class_weight=None), param, cv = 4,n_jobs=4,scoring='roc_auc')

cvelasticnet = sgdElasticAUCnow.fit(X_train,y_train)
cvelasticnet.best_params_



In [ ]:
elasticnet = SGDClassifier(loss="log", penalty="elasticnet", alpha=0.05,l1_ratio=0.25,class_weight=None)
# Apprentissage du modele
elasticnet.fit(X_train,y_train)
elasticnet.coef_
  #compute lift at 10%#

### Analyser les résultats et les performances


### PARAMETRER LE MODELE RANDOMFOREST
Tester un modle light peu profond et un modele complexe

In [ ]:
# exemple randomforest
rf_light = RandomForestClassifier(n_estimators = 
                            criterion = "gini",
                                       max_features = ,
                                       max_depth = ,
                                       min_samples_leaf = ,                                       
                                       bootstrap = True,
                                       n_jobs = -1, # coeurs
                                       random_state = 1234,
                                       class_weight="balanced"
                                     )    
# Apprentissage du modele avec methode fit

In [ ]:
# Apprentissage du modele avec methode fit
rf_light.fit(X_train,y_train)
rf_complex.fit(X_train,y_train)

### ANALYSER LES RESULTATS

In [ ]:
# Importance des variables
df=pd.DataFrame(rf.feature_importances_,X.columns.values)
df.columns=['Importance']
df.sort_values(by='Importance',ascending=False)

In [ ]:
# Prediction des probabilités de 1 ( array2d ) avec fonction predict_proba


In [ ]:
#AUC sur train et test avec roc_auc_score

In [ ]:
#compute lift at 10%#
from fonctions_metrics import lift
lift(probas_train,X_train,y_train)
lift(probas_test,X_test,y_test)

In [ ]:

#compute lift at 5%
lift(probas_test,X_test,y_test,p=5)

In [ ]:
# métriques (liste de dictionnaires)
metriques = metriques+[{'model':'rf','AUC_test':round(roc_auc_score(y_test,probas_test),2),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}]

# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme


### PARAMETRER LE MODELE BOOSTING


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
########### sans randomisation (subsample=1.0 = on prend tt l'échantillon)

In [ ]:
gbt_noRand05=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=500,
                           subsample=1.0, min_samples_split=20, min_samples_leaf=10,
                           max_depth=4)

In [ ]:
# Apprentissage du modele
gbt_noRand05.fit(X_train,y_train)

In [ ]:
# étude du nb iterations optimal en affichant la fonction de perte sur lech test

In [ ]:
niter=500
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_noRand05.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_noRand05.loss_(y_test, y_pred)

plt.figure(figsize=(8, 6))
# Erreur sur le test (evolution deviance)
plt.plot(iter,test_deviance,label='Test',color='darkorange')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_noRand05.train_score_,label='Apprentissage',color='navy')    
# Diminution de l'erreur rapport modele precedant (par rapport au oob)
#plt.plot(iter,gbt_noRand05.oob_improvement_)
plt.legend(loc="upper right", fontsize=12)


In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = gbt_noRand05.predict_proba(X_test)[:,1]
probas_train = gbt_noRand05.predict_proba(X_train)[:,1]
  #AUC
roc_auc_score(y_train,probas_train)
# 1
roc_auc_score(y_test,probas_test)
# AUC 0.91 sur test

#compute lift at 10%#
lift(probas_train,X_train,y_train)
lift(probas_test,X_test,y_test)
# x 7.6
#compute lift at 5%
lift(probas_test,X_test,y_test,p=5)
# 10.36

In [ ]:
########### algo avec randomisation subsample=0.5 et max_feature = racine(nb variable)
###########
gbt_Rand=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=500,
                           subsample=0.5, min_samples_split=30, min_samples_leaf=20,
                           min_weight_fraction_leaf=0.005,
                           max_depth=3,max_leaf_nodes=12,max_features="sqrt")
# Apprentissage du modele
gbt_Rand.fit(X_train,y_train)

In [ ]:
niter=500
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred in enumerate(gbt_Rand.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    test_deviance[i] = gbt_Rand.loss_(y_test, y_pred)

# negative cumulative sum of oob improvements (améliration de l'erreur)
#out-of-bag (OOB) estimates can be a useful heuristic to estimate the “optimal” number of boosting iterations. OOB estimates are almost identical to cross-validation estimates but they can be computed on-the-fly without the need for repeated model fitting. OOB estimates are only available for Stochastic Gradient Boosting (i.e. subsample < 1.0), the estimates are derived from the improvement in loss based on the examples not included in the bootstrap sample (the so-called out-of-bag examples). The OOB estimator is a pessimistic estimator of the true test loss, but remains a fairly good approximation for a small number of trees.
cumsum = -np.cumsum(gbt_Rand.oob_improvement_)

In [ ]:

plt.figure(figsize=(8, 6))

plt.plot(iter,test_deviance,label='Test',color='darkorange')
plt.plot(iter,gbt_Rand.train_score_,label='Apprentissage',color='navy')    
plt.figure(figsize=(8, 6))
plt.plot(iter,cumsum,label='OOB loss')
plt.figure(figsize=(8, 6))

plt.plot(iter,gbt_Rand.oob_improvement_,label='amélioration loss sur OOB')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_noRand05.train_score_,label='Apprentissage',color='navy')    

 

In [ ]:
       # min vers 100
     #   pred_at_eachstage=enumerate(gbt_Rand.staged_predict(X_testCR),100)
gbt_Rand=GradientBoostingClassifier(loss='deviance', learning_rate=0.05,
                           n_estimators=150,
                           subsample=0.5, min_samples_split=30, min_samples_leaf=20,
                           min_weight_fraction_leaf=0.005,
                           max_depth=3,max_leaf_nodes=12,max_features="sqrt")
# Apprentissage du modele
gbt_Rand.fit(X_train,y_train)


probas_test = gbt_Rand.predict_proba(X_test)[:,1]
probas_train = gbt_Rand.predict_proba(X_train)[:,1]
  #AUC
roc_auc_score(y_train,probas_train)
# 0,979
roc_auc_score(y_test,probas_test)
# AUC 0.89 sur test

#compute lift at 10%#
lift(probas_train,X_train,y_train)
lift(probas_test,X_test,y_test)
# x 7.3
#compute lift at 5%
lift(probas_test,X_test,y_test,p=5)
# 9.1